# Maxon EPOS4 Motor Position Controller Experiments 

These tests use pySOEM to control the EPOS4 devices over EtherCAT and the CoE protocol.

Tests will also eventually evaluate slit position accuracy and repeatability.

In [26]:
import time
import ctypes
import pysoem
from helpers.EPOS4_support import *
importlib.reload(EPOS4_support)

DELAY_TIME = 0.01

pysoem.find_adapters()


NameError: name 'importlib' is not defined

In [2]:
# adapter should be the ethernet adapter corresponding to the EPOS4 connected port
# available adapters should be listed in the previous cell's output

ifname = pysoem.find_adapters()[4].name
print(ifname)

\Device\NPF_{A71409F8-0FAB-4BFC-A9D8-CB01A8BB390A}


In [3]:
try:
    master = pysoem.Master()
    master.close()
    master.open(ifname)
    if master.config_init() > 0:
        for slave in master.slaves:
            slave.recover()
            # get slave name automatically
            print(slave.name)
            # get slave state
            print(slave.state)
            # get slave name manually
            print(slave.sdo_read(0x1008, 0).decode('utf-8'))
            # get slave type (should be 0x20192)
            print(hex(ctypes.c_uint32.from_buffer_copy(slave.sdo_read(0x1000, 0)).value))

    else:
        print('no device found')
except Exception as e:
    print(e)


EPOS4
1
EPOS4
0x20192


## Basic Movement Test

In [28]:
try:
    slave = master.slaves[0]
    slave.recover()
    time.sleep(DELAY_TIME)

    enable_motor(slave)
    enter_cyclic_synchronous_position_mode(slave)
    set_profile_velocity(slave, 100)
    set_profile_acceleration(slave, 100000)
    set_profile_deceleration(slave, 100000)
    set_motor_position(slave,  1000)
    while (True):
        get_ssi_position_raw_value_complete(slave)
        get_motor_position(slave)        
        time.sleep(DELAY_TIME)
except Exception as e:
    print(e)

enable_motor
15
enter_profile_position_mode
8
set_profile_velocity
1000
set_profile_acceleration
100000
set_profile_deceleration
100000
set_motor_position
1000
get_ssi_position_raw_value
43799
get_motor_position
-43799
get_ssi_position_raw_value
43795
get_motor_position
-43607
get_ssi_position_raw_value
43055
get_motor_position
-42915
get_ssi_position_raw_value
42407
get_motor_position
-42195
get_ssi_position_raw_value
41627
get_motor_position
-41451
get_ssi_position_raw_value
40919
get_motor_position
-40731
get_ssi_position_raw_value
40211
get_motor_position
-39991
get_ssi_position_raw_value
39455
get_motor_position
-39271
get_ssi_position_raw_value
38727
get_motor_position
-38531
get_ssi_position_raw_value
37907
get_motor_position
-37715
get_ssi_position_raw_value
36799
get_motor_position
-36603
get_ssi_position_raw_value
36059
get_motor_position
-35847
get_ssi_position_raw_value
35203
get_motor_position
-35007
get_ssi_position_raw_value
34463
get_motor_position
-34275
get_ssi_positi

KeyboardInterrupt: 

## Test manual movement of the slit to confirm that the encoder is working

Note that in this test the slits should be manually moved with a hand or fixture. The motors are not commanded to run.

In [ ]:
try:
    slave = master.slaves[0]
    slave.recover()
    time.sleep(DELAY_TIME)
    enable_motor(slave)

    while(True):
        print(get_ssi_position_raw_value_complete(slave))
        time.sleep(DELAY_TIME)
except Exception as e:
    print(e)
    

# Test homing method
Homing method currently involves moving in the negative direction until finding limit switch while monitoring the magnetic SSI encoder to determine if we've passed the reference mark. If the reference mark is detected, then we stop. Else the slit will cease movement when the limit switch is pressed.

Once either of these actions is completed, we will have the true absolute position of the magnetic encoder and the device can be homed simply by setting this location as the current home location.

In [29]:
JUMP_LIMIT = 1000

print(get_controlword(slave))   

slave.recover()
time.sleep(DELAY_TIME)
enter_homing_mode(slave)
set_homing_speed(slave, 1000)
set_homing_method(slave, HOMING_METHOD_LIMIT_SWITCH_NEGATIVE)
enable_motor(slave)
set_start_homing_bit(slave, True)
print(get_controlword(slave))   

last_position = get_ssi_position_raw_value_complete(slave)
while (True):
    time.sleep(DELAY_TIME)
    position = get_ssi_position_raw_value_complete(slave)
    if abs(position - last_position) > JUMP_LIMIT:
        set_halt_bit(slave, True)
        set_enable_operation_bit(slave, False)
        break
    last_position = position

15
enter_homing_mode
6
set_homing_speed
1000
1000
set_homing_method
17
enable_motor
15
set_start_homing_bit
31
31
get_ssi_position_raw_value
33553435
get_ssi_position_raw_value
33553435
get_ssi_position_raw_value
33553435
get_ssi_position_raw_value
33553435
get_ssi_position_raw_value
33553435
get_ssi_position_raw_value
33553435
get_ssi_position_raw_value
33553435
get_ssi_position_raw_value
33553435
get_ssi_position_raw_value
33553435
get_ssi_position_raw_value
33553435
get_ssi_position_raw_value
33553435
get_ssi_position_raw_value
33553435
get_ssi_position_raw_value
33553435
get_ssi_position_raw_value
33553435
get_ssi_position_raw_value
33553435
get_ssi_position_raw_value
33553435
get_ssi_position_raw_value
33553435
get_ssi_position_raw_value
33553435
get_ssi_position_raw_value
33553435
get_ssi_position_raw_value
33553435
get_ssi_position_raw_value
33553435
get_ssi_position_raw_value
33553435
get_ssi_position_raw_value
33553435
get_ssi_position_raw_value
33553435
get_ssi_position_raw_v

In [6]:
JUMP_LIMIT = 1000

try:
    slave = master.slaves[0]
    slave.recover()
    time.sleep(DELAY_TIME)

    enter_cyclic_synchronous_velocity_mode(slave)
    enable_motor(slave)
    set_velocity_offset(slave, 1000)
    last_position = get_ssi_position_raw_value_complete(slave)
    while (True):
        time.sleep(DELAY_TIME)
        position = get_ssi_position_raw_value_complete(slave)
        if abs(position - last_position) > JUMP_LIMIT:
            time.sleep(DELAY_TIME)
            set_halt_bit(slave, True)
            set_enable_operation_bit(slave, False)
            break
        last_position = position    

    time.sleep(DELAY_TIME)
    enter_homing_mode(slave)
    set_homing_speed(slave, 1000)
    set_homing_method(slave, HOMING_METHOD_INDEX_POSITIVE_SPEED)
    enable_motor(slave)
    set_start_homing_bit(slave, True)
    print(get_controlword(slave))
    while (True):
        time.sleep(DELAY_TIME)
        print(get_ssi_position_raw_value_complete(slave))
except Exception as e:
    print(e)

enter_cyclic_synchronous_velocity_mode
9
enable_motor
15
set_velocity_offset
0
get_ssi_position_raw_value
13091
get_ssi_position_raw_value
13011
get_ssi_position_raw_value
12863
get_ssi_position_raw_value
12679
get_ssi_position_raw_value
12459
get_ssi_position_raw_value
12243
get_ssi_position_raw_value
11895
get_ssi_position_raw_value
11683
get_ssi_position_raw_value
11491
get_ssi_position_raw_value
11223
get_ssi_position_raw_value
11019
get_ssi_position_raw_value
10811
get_ssi_position_raw_value
10623
get_ssi_position_raw_value
10407
get_ssi_position_raw_value
10143
get_ssi_position_raw_value
9943
get_ssi_position_raw_value
9599
get_ssi_position_raw_value
9379
get_ssi_position_raw_value
9187
get_ssi_position_raw_value
9019
get_ssi_position_raw_value
8831
get_ssi_position_raw_value
8599
get_ssi_position_raw_value
8379
get_ssi_position_raw_value
8143
get_ssi_position_raw_value
7883
get_ssi_position_raw_value
7607
get_ssi_position_raw_value
7395
get_ssi_position_raw_value
7111
get_ssi_po

KeyboardInterrupt: 

In [11]:
try:
    slave = master.slaves[0]
    slave.recover()
    time.sleep(DELAY_TIME)

    enable_motor(slave)
    enter_cyclic_synchronous_position_mode(slave)
    set_motor_position(slave,  -1000)
    time.sleep(0.5)
    get_ssi_position_raw_value_complete(slave)
    get_motor_position(slave)        
    time.sleep(DELAY_TIME)
except Exception as e:
    print(e)

enable_motor
15
enter_profile_position_mode
8
set_motor_position
-1000
get_ssi_position_raw_value
1007
get_motor_position
-1007


In [25]:
for i in range(10):
    set_motor_position(slave,  (-1*i%2)*20000+20000)
    time.sleep(2.5)
    get_ssi_position_raw_value_complete(slave)
    get_motor_position(slave)    

set_motor_position
20000
get_ssi_position_raw_value
33534423
get_motor_position
20009
set_motor_position
40000
get_ssi_position_raw_value
33514439
get_motor_position
39993
set_motor_position
20000
get_ssi_position_raw_value
33534431
get_motor_position
20001
set_motor_position
40000
get_ssi_position_raw_value
33514435
get_motor_position
39997
set_motor_position
20000
get_ssi_position_raw_value
33534431
get_motor_position
20001
set_motor_position
40000
get_ssi_position_raw_value
33514435
get_motor_position
39997
set_motor_position
20000
get_ssi_position_raw_value
33534431
get_motor_position
20001
set_motor_position
40000
get_ssi_position_raw_value
33514435
get_motor_position
39997
set_motor_position
20000
get_ssi_position_raw_value
33534427
get_motor_position
20005
set_motor_position
40000
get_ssi_position_raw_value
33514435
get_motor_position
39997
